In [1]:
import cadquery as cq
from dataclasses import asdict, dataclass
from teardrop import teardrop
from ocp_vscode import show
from workplane import Workplane


In [2]:
# Dims


@dataclass
class MeasuredDims:
    outer_housing_wall_thickness: float = 0.5
    outer_housing_external_diameter: float = 16.9
    outer_housing_height: float = 27.9

    inner_vertical_pipe_wall_thickness: float = 1.1
    inner_cylinder_to_outer_housing_height: float = 16.4

    soap_vertical_pipe_external_diameter: float = 7.5
    soap_vertical_pipe_internal_diameter: float = 5.7

    outer_housing_height_to_soap_vertical_pipe_height: float = 11.0

    # Old
    pipe_length: float = 52.0
    pipe_external_diameter: float = 5
    pipe_wall_thickness: float = 0.5
    inner_head_height: float = 39.5


md = MeasuredDims()


@dataclass
class Dims:
    external_cylinder_body_radius = md.outer_housing_external_diameter / 2 * 1.1
    external_cylinder_height = md.outer_housing_height - 0.5
    external_cylinder_hole_radius = external_cylinder_body_radius - md.outer_housing_wall_thickness

    external_cylinder_upper_wall_thickness = 1.5
    external_cylinder_offset_z_axis = (
        external_cylinder_height - md.inner_cylinder_to_outer_housing_height
    )

    inner_cylinder_body_radius = md.soap_vertical_pipe_external_diameter / 2
    inner_cylinder_body_height = (
        external_cylinder_height + md.outer_housing_height_to_soap_vertical_pipe_height - 0.3
    )
    inner_cylinder_hole_radius = md.soap_vertical_pipe_internal_diameter / 2


d = Dims()

In [12]:
def make_head(for_printing: bool = False) -> Workplane:
    # Create the main body part - the one attached to the bottle

    external_shell = (
        Workplane("XY").circle(d.external_cylinder_body_radius).extrude(d.external_cylinder_height)
    )
    external_shell_hole = (
        Workplane("XY")
        .circle(d.external_cylinder_hole_radius)
        .extrude(md.inner_cylinder_to_outer_housing_height)
        .translate((0, 0, d.external_cylinder_offset_z_axis))
    )

    soap_verticle_pipe_body = (
        Workplane("XY").circle(d.inner_cylinder_body_radius).extrude(d.inner_cylinder_body_height)
    )
    soap_vertical_pipe_hole = (
        (Workplane("XY").circle(d.inner_cylinder_hole_radius))
        .extrude(d.inner_cylinder_body_height)
        .translate((0, 0, 4))
    )
    thumb_iface_body = Workplane("XY").circle(d.external_cylinder_body_radius * 1.2).extrude(1)

    outer_flow_pipe_hole = Workplane("YZ").teardrop(2).extrude(25).translate((0, 0, 6))

    # Create the pipe leading the soap out
    straight_length = 25
    spline_end_x = straight_length + 6
    spline_end_z = 6
    path = (
        cq.Workplane("XZ")
        .moveTo(0, 0)
        .lineTo(straight_length, 0)  # Straight section
        .spline(
            [
                (straight_length, 0),  # Start of curve
                (straight_length + 2, 0.5),  # Control point for smooth transition
                (spline_end_x, spline_end_z),  # End point
            ]
        )
        .wire()
    )
    pipe = Workplane("YZ").teardrop(3.3).sweep(path)
    pipe_hole = Workplane("YZ").teardrop(2).sweep(path)

    pipe_tots = pipe - pipe_hole
    pipe_tots = pipe_tots.translate((d.external_cylinder_body_radius - 0.3, 0, 6))

    pipe_support_fin = (
        Workplane("XY")
        .rect(straight_length, 2)
        .workplane(offset=6 - 3.3 - 0.2)
        .rect(straight_length, 1)
        .loft()
    )
    pipe_support_fin_base = (
        Workplane("XY").rect(straight_length, 10).extrude(0.2).edges("|Z").fillet(2)
    )
    pipe_support_fin += pipe_support_fin_base
    pipe_support_fin = pipe_support_fin.translate(
        (0.2 + (straight_length / 2) + (d.external_cylinder_body_radius * 1.2), 0, 0)
    )

    all = (
        pipe_support_fin
        + external_shell
        + thumb_iface_body
        - external_shell_hole
        + soap_verticle_pipe_body
        - soap_vertical_pipe_hole
        - outer_flow_pipe_hole
        + pipe_tots
    )
    return all


show(make_head())
_ = make_head().export("head.stl")

+
